In [ ]:
# Load raw data
import os
import sys
module_path = os.path.abspath('/home/mjo24/Projects/WildSemiFusion/')
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

import src.config as cfg
import matplotlib.pyplot as plt
from cv2 import imread
import matplotlib
import numpy as np

# Load train data
# train_image_files = os.listdir(f"{cfg.TRAIN_DIR}/pylon_camera_node")
# train_semantic_images = os.listdir(f"{cfg.TRAIN_DIR}/pylon_camera_node_label_id")
# train_rgb_images = [image_to_array(f"{cfg.TRAIN_DIR}/pylon_camera_node/{image_file}") for image_file in train_image_files]
image_path = os.path.join(cfg.BASE_DIR, "../input/rellis_2d_preprocessed/train/00000/pylon_camera_node/frame000308-1581624683_550.jpg")
# print(os.path.abspath(image_path))
image = imread(image_path)
print(matplotlib.__version__)
print(np.__version__)
print(image)
plt.imshow(image)
# train_semantics = [image_to_array(f"{cfg.TRAIN_DIR}/pylon_camera_node_label_id/{label_file}") for label_file in train_semantic_images]
# train_preloaded_data = {'rgb_images': train_rgb_images, 'gt_semantics': train_semantics}

# # Load val data
# val_image_files = os.listdir(f"{cfg.VAL_DIR}/pylon_camera_node")
# val_semantic_images = os.listdir(f"{cfg.VAL_DIR}/pylon_camera_node_label_id")
# val_rgb_images = [image_to_array(f"{cfg.VAL_DIR}/pylon_camera_node/{image_file}") for image_file in val_image_files]
# val_semantics = [image_to_array(f"{cfg.VAL_DIR}/pylon_camera_node_label_id/{label_file}") for label_file in val_semantic_images]
# val_preloaded_data = {'rgb_images': val_rgb_images, 'gt_semantics': train_semantics}

In [ ]:
# Load dataset
import torch
from torch.utils.data import DataLoader
from src.data.rellis_2D_dataset import Rellis2DDataset
from src.models.model_1 import MultiModalNetwork

# Setup dataset
train_dataset = Rellis2DDataset(preloaded_data=train_preloaded_data, num_bins=cfg.NUM_BINS, image_size=cfg.IMAGE_SIZE, image_noise=cfg.IMAGE_NOISE, image_mask_rate=cfg.IMAGE_MASK_RATE)
# val_dataset = Rellis2DDataset(preloaded_data=val_preloaded_data, num_bins=cfg.NUM_BINS, image_size=cfg.IMAGE_SIZE, image_noise=cfg.IMAGE_NOISE, image_mask_rate=cfg.IMAGE_MASK_RATE)
print("Datasets created successfully")

# Load dataset
train_dataloader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=cfg.NUM_WORKERS, pin_memory=False, drop_last=True)
# val_dataloader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False, num_workers=cfg.NUM_WORKERS, pin_memory=False, drop_last=True)
print("DataLoaders created successfully")

In [ ]:
# Verify dataset
import matplotlib.pyplot as plt
from src.data.utils.data_processing import lab_discretized_to_rgb

count = 0 
for idx in range(len(train_dataset)):
    print("Original RGB Shape: ", train_rgb_images[idx].shape)

    sample = train_dataset[idx]
    gt_semantics = sample['gt_semantics']
    gt_color = sample['gt_color']
    lab_image = sample['lab_image']
    gray_image = sample['gray_image']

    fig, axs = plt.subplots(3, 2, figsize=(15, 15))
    print(f"GT Semantics Shaep: {gt_semantics.numpy().shape}")
    print(f"GT Color Shape: {gt_color.numpy().shape}")
    print(f"LAB Image Shape: {lab_image.numpy().shape}")
    print(f"Gray Image Shape: {gray_image.numpy().shape}")
    axs[0, 0].imshow(lab_discretized_to_rgb(gt_color.numpy(), cfg.NUM_BINS))
    axs[0, 0].set_title('GT LAB Image to RGB')
    axs[0, 1].imshow(lab_discretized_to_rgb(lab_image.numpy().transpose(1, 2, 0), cfg.NUM_BINS))
    axs[0, 1].set_title('Masked Noisy LAB Image to RGB')


    axs[1, 0].imshow(gray_image.numpy().transpose(1, 2, 0), cmap='gray')
    axs[1, 0].set_title('Gray Image')
    axs[1, 1].imshow(train_rgb_images[0])
    axs[1, 1].set_title('Original RGB Image')

    axs[2, 0].imshow(gt_semantics.to(torch.uint8).numpy())
    axs[2, 0].set_title('GT Semantics')

    plt.tight_layout()
    plt.show()

    break